# 1) 코사인 유사도

In [ ]:
# Numpy를 사용해서 코사인 유사도를 계산하는 함수를 구현하고 각 문서 벡터 간의 코사인 유사도 계산
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
  return dot(A, B) / (norm(A)*norm(B))

doc1 = np.array([0, 1, 1, 1])
doc2 = np.array([1, 0, 1, 1])
doc3 = np.array([2, 0, 2, 2])

print("문서 1과 문서 2의 유사도 :", cos_sim(doc1, doc2))
print("문서 1과 문서 3의 유사도 :", cos_sim(doc1, doc3))
print("문서 2와 문서 3의 유사도 :", cos_sim(doc2, doc3))

문서 1과 문서 2의 유사도 : 0.6666666666666667
문서 1과 문서 3의 유사도 : 0.6666666666666667
문서 2와 문서 3의 유사도 : 1.0000000000000002


In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('/content/sample_data/movies_metadata.csv',low_memory=False)
data.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [10]:
# 상위 2만개의 샘플을 data에 저장
data = data.head(20000)

In [11]:
# overview 열에 존재하는 모든 결측값(null)을 전부 카운트하여 출력
print('overview 열의 결측값의 수 :', data['overview'].isnull().sum())

overview 열의 결측값의 수 : 135


In [12]:
# 결측값을 빈 값으로 대체
data['overview'] = data['overview'].fillna('')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [13]:
# TF-IDF 행렬을 구한 후 행렬의 크기 출력
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])
print('TF-IDF 행렬의 크기(shape) :', tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape) : (20000, 47487)


In [14]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :', cosine_sim.shape)

코사인 유사도 연산 결과 : (20000, 20000)


In [15]:
title_to_index = dict(zip(data['title'], data.index))

# 영화 제목 Father of the Bride Part II의 인덱스를 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)

4


In [16]:
# 선택한 영화의 제목을 입력하면 코사인 유사도를 통해 가장 overview가
# 유사한 10개의 영화를 찾아내는 함수
def get_recommendations(title, cosine_sim=cosine_sim):
  # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다
  idx = title_to_index[title]

  # 해당 영화와 모든 영화와의 유사도를 가져온다
  sim_scores = list(enumerate(cosine_sim[idx]))

  # 유사도에 따라 영화들을 정렬한다
  sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)

  # 가장 유사한 10개의 영화를 받아온다
  sim_scores = sim_scores[1:11]

  # 가장 유사한 10개의 영화의 인덱스를 얻는다
  movie_indices = [idx[0] for idx in sim_scores]

  # 가장 유사한 10개의 영화의 제목을 리턴한다
  return data['title'].iloc[movie_indices]

In [17]:
# 영화 다크 나이트 라이즈와 overview가 유사한 영화
get_recommendations('The Dark Knight Rises')

12481                            The Dark Knight
150                               Batman Forever
1328                              Batman Returns
15511                 Batman: Under the Red Hood
585                                       Batman
9230          Batman Beyond: Return of the Joker
18035                           Batman: Year One
19792    Batman: The Dark Knight Returns, Part 1
3095                Batman: Mask of the Phantasm
10122                              Batman Begins
Name: title, dtype: object

# 2) 여러가지 유사도 기법
## 1. 유클리드 거리

In [19]:
# 유클리드 거리는 자카드 유사도나 코사인 유사도만큼 유용한 방법은 아니다
import numpy as np

def dist(x, y):
  return np.sqrt(np.sum((x-y)**2))

doc1 = np.array((2, 3, 0, 1))
doc2 = np.array((1, 2, 3, 1))
doc3 = np.array((2, 1, 2, 2))
docQ = np.array((1, 1, 0, 1))

# 유클리드 거리 값이 작을수록 거리가 가깝다. 즉, 유사하다는 것 을 알 수 있다
print("문서 1과 문서 Q의 거리 :", dist(doc1, docQ))
print("문서 2와 문서 Q의 거리 :", dist(doc2, docQ))
print("문서 3과 문서 Q의 거리 :", dist(doc3, docQ))

문서 1과 문서 Q의 거리 : 2.23606797749979
문서 2와 문서 Q의 거리 : 3.1622776601683795
문서 3과 문서 Q의 거리 : 2.449489742783178


## 2. 자카드 유사도

In [20]:
doc1 = "apple banana everyone like likey watch card holder"
doc2 = "apple banana coupon passport love you"

# 토큰화
tokenized_doc1 = doc1.split()
tokenized_doc2 = doc2.split()

print('문서1 :',tokenized_doc1)
print('문서2 :',tokenized_doc2)

문서1 : ['apple', 'banana', 'everyone', 'like', 'likey', 'watch', 'card', 'holder']
문서2 : ['apple', 'banana', 'coupon', 'passport', 'love', 'you']


In [21]:
# 문서 1과 2의 합집합
union = set(tokenized_doc1).union(set(tokenized_doc2))
print('문서 1과 문서 2의 합집합 :', union)

문서 1과 문서 2의 합집합 : {'watch', 'love', 'banana', 'everyone', 'apple', 'likey', 'card', 'like', 'holder', 'passport', 'coupon', 'you'}


In [23]:
# 문서 1과 2의 교집합
intersection = set(tokenized_doc1).intersection(set(tokenized_doc2))
print('문서 1과 문서 2의 교집합 :', intersection)

문서 1과 문서 2의 교집합 : {'apple', 'banana'}


In [24]:
# 교집합의 크기를 합집합의 크기로 나누기(자카드 유사도)
print('자카드 유사도 :', len(intersection)/len(union))

자카드 유사도 : 0.16666666666666666
